<h1> Read the training data <h1>

In [ ]:
lines = open('train.tsv','r',encoding='utf-8').readlines()

In [ ]:
len(lines)

In [ ]:
d =lines[0].strip().split('\t')
for i in range(0,len(d)):
    print (i,d[i])

<h1> Label 1 for true & mostly true , Label 0 for fake or partially true statements </h1>

In [ ]:
true_data = []
false_data = []
overall_data = []
label = []
for i in range(0,len(lines)):
    line = lines[i].strip().lower().split()
    overall_data.append(lines[i].strip().lower())
    if (line[1]== 'true' or line[1] =='mostly-true'):
        true_data.append(lines[i].strip().lower())
        label.append(1)
    else:
        false_data.append(lines[i].strip().lower())
        label.append(0)
## Please note overall data still contains all of training data

## Please note overall data still contains all of training data



<h1>use nltk word tokenize and remove stop words </h1>



In [2]:
##use nltk word tokenize and remove stop words
data = []
for line in vocabulary:
    text = vocabulary['words']
    party = line.split('\t')[7].lower()
    line = party + ' ' + text 
    tokens = [i for i in word_tokenize(line.strip().lower()) if i not in stop]
    bigrams = []
    for i in range(0,len(tokens)-1):
        bigram = tokens[i]+'-'+tokens[i+1]
        bigrams.append(bigram)
    for bigram in bigrams:
        tokens.append(bigram)
        
    data.append(tokens)
data[0]

NameError: name 'vocabulary' is not defined

<h1> Use Gensim and create word vectors from the dataset </h1>

In [ ]:
# code referenced from https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
from gensim.models import Word2Vec
model = Word2Vec(data, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
#print(words)
# access vector for one word
print(model['democrat'])
# save model
model.save('overall_model.bin')
# load model
overall_model = Word2Vec.load('overall_model.bin')
print(overall_model)

## Use word vectors as features 
## sentence made of multiple words, then we simply add the vectors of each word and get single vector for that sentence and use that as feature

In [ ]:
X = []
for line in data:
    vectors = []
    for word in line:
        vector = overall_model[word]
        vectors.append(vector)
    word_vector = []

    for i in range(0,len(vectors[0])):
        val = 0.0
        for j in range(0,len(vectors)):
            val += vectors[j][i]
        word_vector.append(val)
    X.append(word_vector)
        


In [ ]:
import pandas as pd
X_df = pd.DataFrame(X)
Y = pd.DataFrame(label)
scaled_X = X_df
scaled_Y = Y
from sklearn.model_selection import train_test_split
from sklearn import linear_model
X_train, X_test, y_train, y_test = train_test_split(scaled_X, scaled_Y, test_size=0.2, random_state=0)
lm = linear_model.LogisticRegression(verbose=1)
model = lm.fit(X_train, y_train)
print (model)
predictions = lm.predict(X_test)

print ("Score:", model.score(X_test, y_test))

## Logistic regression was trained on word vectors and it has accuracy of 0.64 in predicting the fake news on test set. let us test on sample sentence 
lets say if republican says "obama supports third trimester abortion"

In [ ]:
testing= "republican obama supports third trimester abortion"

import numpy as np
def getVector(s):
    s = s.lower()
    words = word_tokenize(s)
    vectors = []
    for word in words:
        if word in overall_model:
            vectors.append(overall_model[word])
    sum_vector = []
    for i in range(0,len(vectors[0])):
        val = 0.0
        for j in range(0,len(vectors)):
            val += vectors[j][i]
        sum_vector.append(val)
    return pd.DataFrame([sum_vector])

testX = getVector(testing)
result_score = model.predict(testX)
model.predict_proba(testX)
### 0.71 probability that news belong to class 2 which is fake news

## Class 1 probability is 0.28 and class 2 probability is 0.71 , so its likely a fake news